In [6]:
# 2299x1010

DATA_INICIO = '2018-01-01 00:00:00'
DATA_FIM = '2025-12-31 23:59:59'
PALAVRAS_CHAVE = ["Vlr", "Valor", "vlr", "valor", "Vr", "vr", 
                  "base", "Base", "Salário", "Salario", "salário", 
                  "salario", "Provento", "provento", "Desconto", "desconto", 
                  "diferenca", "Diferenca", "diferença", "Diferença"]

In [7]:
# 2299x1010

import pandas as pd
import pyperclip
import os
import numpy as np

a = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\gabarito\\2299')
b = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\gabarito\\1010')

print(len(a))
print(len(b))

8280
3646


In [8]:
# 2299x1010

relatorios = ['2299', '1010']

sys_df_1010 = None
sys_df_2299 = None
df_sys = None
parquet_path_sys = None

for relatorio in relatorios:
    relatorio_sis = f's{relatorio}'

    try:
        parquet_path_sys = f'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\{relatorio_sis}\\'

        df_sys = pd.read_parquet(parquet_path_sys, engine='pyarrow')

        df_sys['periodo'] = (
                df_sys['id'].astype(str).str[17:21] + '-' +
                df_sys['id'].astype(str).str[21:23] + '-' +
                df_sys['id'].astype(str).str[23:25] + ' ' +
                df_sys['id'].astype(str).str[25:27] + ':' +
                df_sys['id'].astype(str).str[27:29]
        )

        df_sys['periodo'] = pd.to_datetime(df_sys['periodo'])
        dt_ini = pd.to_datetime(DATA_INICIO)
        dt_fim = pd.to_datetime(DATA_FIM)

        if relatorio != '1010':
            df_sys = df_sys[(df_sys['periodo'] >= dt_ini) & (df_sys['periodo'] <= dt_fim)]

    except Exception as e:
        print(f"Erro ao carregar {parquet_path_sys}: {e}")

    if relatorio == '1010':
        sys_df_1010 = df_sys
    elif relatorio == '2299':
        sys_df_2299 = df_sys

# Renomeia colunas
s2299 = sys_df_2299.copy().reset_index()
s1010 = sys_df_1010.copy()

In [9]:
# 2299x1010

s2299.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificador_tabela_rubricas': 'id_rubrica',
            'periodo': 'periodo_2299',
            'id': 'id_s2299'
        }, inplace=True)

s1010.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificacao_rubrica': 'id_rubrica',
            'periodo': 'periodo_1010',
            'id': 'id_1010'
        }, inplace=True)

s2299['periodo_2299'] = pd.to_datetime(s2299['periodo_2299'])
s1010['periodo_1010'] = pd.to_datetime(s1010['periodo_1010'])

# 1ª Consolidação: período_1010 <= período
merged = s2299.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('_s2299', '_1010')
        )

# Filtrar por datas válidas
valid = merged[merged['periodo_1010'] <= merged['periodo_2299']]

# Agrupar por ID completo + rubrica, e pegar o idxmax
idx = valid.groupby(['id_s2299', 'cod_rubrica', 'id_rubrica', 'index'])['periodo_1010'].idxmax()

# Selecionar as linhas finais
s2299_final = valid.loc[idx].copy()

colunas_s2299 = [col for col in sys_df_2299.columns]

s2299_final.rename(columns={
            'id_s2299': 'id',
            'periodo_2299': 'periodo',
            'id_rubrica': 'identificador_tabela_rubricas',
            'cod_rubrica': 'codigo_rubrica',
            'numero_do_recibo_s2299': 'numero_do_recibo',
            'identificacao_ambiente_s2299': 'identificacao_ambiente',
            'processo_emissao_s2299': 'processo_emissao',
            'versao_aplicativo_s2299': 'versao_aplicativo',
            'tipo_inscricao_empregador_s2299': 'tipo_inscricao_empregador',
            'numero_inscricao_empregador_s2299': 'numero_inscricao_empregador',
            'observacao_s2299': 'observacao'
        }, inplace=True)

# Juntar com as 3 colunas desejadas do s1010
colunas_s2299 = [col for col in colunas_s2299 if col not in ['id']]

colunas_final = ['id', 'id_1010', 'index'] + colunas_s2299 + [
            'descricao_rubrica', 'tipo_da_rubrica', 'incidencia_previdencia_social'
        ]

colunas_final = [col for col in colunas_final if col not in ['periodo']]

s2299_final = s2299_final[colunas_final]

s2299_2_ = s2299[~s2299['index'].isin(s2299_final['index'])]

merged = s2299_2_.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010'),
            how='left'
        )

merged_ = merged.rename(columns={
            'periodo_2299': 'periodo',
            'id_s2299': 'id',
            'cod_rubrica': 'codigo_rubrica',
            'id_rubrica': 'identificador_tabela_rubricas'
        }).drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_tabela_rubricas'])

colunas_final = [col for col in colunas_final if col not in ['periodo']]

merged_selected = merged_[colunas_final]

merged_com_s2299_final = pd.concat([s2299_final, merged_selected])

df_final_ = merged_com_s2299_final.drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_tabela_rubricas'])\
    .drop(columns=['index', 'id_1010'], errors='ignore').fillna('')

In [10]:
# 2299x1010

num_linhas, num_colunas = df_final_.shape
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

for coluna in df_final_.columns:
    linhas_preenchidas = df_final_[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in PALAVRAS_CHAVE):
            # Soma
            soma_valores = (
                df_final_[coluna]
                .replace('', np.nan)
                .astype(float)
                .sum(skipna=True)
            )

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")


✅ Resultado copiado para a área de transferência!


C:\Users\GabrielFelipe\AppData\Local\Temp\ipykernel_16708\878967889.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('', np.nan)
